In [7]:
import os
import gradio as gr
from dotenv import load_dotenv
from swarmauri.standard.llms.concrete.OpenAIToolModel import OpenAIToolModel as LLM
from swarmauri.standard.conversations.concrete.Conversation import Conversation
from swarmauri.standard.toolkits.concrete.Toolkit import Toolkit
from swarmauri.standard.agents.concrete.ToolAgent import ToolAgent


In [8]:
load_dotenv()
API_KEY = os.getenv('OPENAPI_API_KEY')

In [9]:
tool_model = LLM(api_key=API_KEY)

ValidationError: 1 validation error for OpenAIToolModel
api_key
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type

### Custom Tool

In [15]:
from typing import List, Literal, Dict
from pydantic import Field
import subprocess as sp
from swarmauri.standard.tools.base.ToolBase import ToolBase
from swarmauri.standard.tools.concrete.Parameter import Parameter


class CustomLinuxTool(ToolBase):
    version: str = "1.0.0"
    # Define the parameters required by the tool
    parameters: List[Parameter] = Field(
        default_factory=lambda: [
            Parameter(
                name="program",
                type="string",
                description="The program that the user wants to open or execute ('gnome-screenshot' or 'baobab' or 'evince')",
                required=True,
                enum=["gnome-screenshot", "baobab", "evince"],
            )
        ]
    )
    name: str = "CustomLinuxTool"
    description: str = "This opens or executes a program based on the user's request."
    type: Literal["CustomLinuxTool"] = "CustomLinuxTool"

    def __call__(self, program) -> Dict[str, str]:
        sp.check_output(program)
        if program == 'gnome-screenshot':
            return {"program": f"Screenshot captured!"}
        return {"program": f"Program Opened: {program}"}

In [16]:
CustomLinuxTool()

CustomLinuxTool(name='CustomLinuxTool', id='11359861-390f-4983-9e85-659218c6be4a', members=[], owner=None, host=None, resource='Tool', version='1.0.0', type='CustomLinuxTool', description="This opens or executes a program based on the user's request.", parameters=[Parameter(name='program', id='465354a4-cf6c-493b-95ce-b08f1750dcba', members=[], owner=None, host=None, resource='Parameter', version='0.1.0', type='string', description="The program that the user wants to open or execute ('gnome-screenshot' or 'baobab' or 'evince')", required=True, enum=['gnome-screenshot', 'baobab', 'evince'])])

In [17]:
toolkit = Toolkit(tools={"CustomLinuxTool": CustomLinuxTool()})

In [18]:
toolkit

Toolkit(name=None, id='08a2b6b5-fdc5-408c-bff5-5ceea9b721dd', members=[], owner=None, host=None, resource='Toolkit', version='0.1.0', type='Toolkit', tools={'CustomLinuxTool': CustomLinuxTool(name='CustomLinuxTool', id='67c17b5f-cd10-400b-93c4-34bbb4a69db2', members=[], owner=None, host=None, resource='Tool', version='1.0.0', type='CustomLinuxTool', description="This opens or executes a program based on the user's request.", parameters=[Parameter(name='program', id='205b9c09-e285-4dbb-bdbd-ba70bd22702f', members=[], owner=None, host=None, resource='Parameter', version='0.1.0', type='string', description="The program that the user wants to open or execute ('gnome-screenshot' or 'baobab' or 'evince')", required=True, enum=['gnome-screenshot', 'baobab', 'evince'])])})

In [19]:
tool_agent = ToolAgent(
    llm=tool_model,
    conversation=Conversation(),
    toolkit=toolkit
)

NameError: name 'tool_model' is not defined

In [20]:
# Define the function to be executed for the Gradio interface
def tool_agent_interface(input_text, history):
    # Directly pass the input_text (string) to tool_agent.exec
    result = tool_agent.exec(input_text)
    return result

# Initialize the Gradio interface
demo = gr.ChatInterface(tool_agent_interface)

if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7865
IMPORTANT: You are using gradio version 4.26.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://3825b84b69382b1a01.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\bigman\anaconda3\envs\swarmauri-0.4.3\Lib\site-packages\pydantic\type_adapter.py", line 277, in _init_core_attrs
    self._core_schema = _getattr_no_parents(self._type, '__pydantic_core_schema__')
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bigman\anaconda3\envs\swarmauri-0.4.3\Lib\site-packages\pydantic\type_adapter.py", line 119, in _getattr_no_parents
    raise AttributeError(attribute)
AttributeError: __pydantic_core_schema__

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\bigman\anaconda3\envs\swarmauri-0.4.3\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 401, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bigman\anaconda3\envs\swarmauri-0.4.3

### Customizing a Tool

In [21]:
from typing import List, Literal, Dict
from pydantic import Field
import subprocess as sp
from swarmauri.standard.tools.base.ToolBase import ToolBase
from swarmauri.standard.tools.concrete.Parameter import Parameter

In [22]:
class CustomGnomeTool(ToolBase): # We name the class
    version: str = "1.0.0"

    # Define the parameters required by the tool
    parameters: List[Parameter] = Field(
        default_factory=lambda: [
            Parameter(
                name="program",
                type="string",
                description="The program that the user wants to open ('gnome-control-center' or 'gnome-calendar' or 'gnome-terminal')", 
                required=True,
                enum=["gnome-control-center", "gnome-calendar", "gnome-terminal"], # We define different values
            )
        ]
    )
    name: str = "CustomGnomeTool" # We change the tool's name attribute
    description: str = "This opens a gnome program based on the user's request." # We change the tool's name attribute
    type: Literal["CustomGnomeTool"] = "CustomGnomeTool" # We change the tool's type Literal, often this will match the name.

    # Finally we customizing the __call__ function
    def __call__(self, program) -> Dict[str, str]:
        sp.check_output(program)
        return {"program": f"Gnome Program Opened: {program}"}

In [23]:
CustomGnomeTool()

CustomGnomeTool(name='CustomGnomeTool', id='63db6a0a-9de8-443b-ac2b-138c95f02f3c', members=[], owner=None, host=None, resource='Tool', version='1.0.0', type='CustomGnomeTool', description="This opens a gnome program based on the user's request.", parameters=[Parameter(name='program', id='2f8552a8-037f-4494-91b9-33942058660e', members=[], owner=None, host=None, resource='Parameter', version='0.1.0', type='string', description="The program that the user wants to open ('gnome-control-center' or 'gnome-calendar' or 'gnome-terminal')", required=True, enum=['gnome-control-center', 'gnome-calendar', 'gnome-terminal'])])

In [24]:
toolkit = Toolkit(tools={"CustomGnomeTool": CustomGnomeTool()})
toolkit

Toolkit(name=None, id='c32e0528-4ea4-4098-bceb-07bed1310479', members=[], owner=None, host=None, resource='Toolkit', version='0.1.0', type='Toolkit', tools={'CustomGnomeTool': CustomGnomeTool(name='CustomGnomeTool', id='dcb7a404-82a0-408f-ad54-57e447cbfdc1', members=[], owner=None, host=None, resource='Tool', version='1.0.0', type='CustomGnomeTool', description="This opens a gnome program based on the user's request.", parameters=[Parameter(name='program', id='1ae34b3b-4d8a-4158-b2ee-50e680d68fc3', members=[], owner=None, host=None, resource='Parameter', version='0.1.0', type='string', description="The program that the user wants to open ('gnome-control-center' or 'gnome-calendar' or 'gnome-terminal')", required=True, enum=['gnome-control-center', 'gnome-calendar', 'gnome-terminal'])])})

In [ ]:
tool_agent = ToolAgent(
    llm=tool_model,
    conversation=Conversation(),
    toolkit=toolkit
)

In [ ]:
# Define the function to be executed for the Gradio interface
def tool_agent_interface(input_text, history):
    # Directly pass the input_text (string) to tool_agent.exec
    result = tool_agent.exec(input_text)
    return result

# Initialize the Gradio interface
demo = gr.ChatInterface(tool_agent_interface)

if __name__ == "__main__":
    demo.launch(share=True)